# TRAINING JUPYTER

### Parameter setting

In [2]:
#! conda install -y tqdm
from utils import Load_FLX_dict
data_folder="Data"
model_folder = "TrainedModels"
log_folder = 'Log'

##### Dictionnary
D = Load_FLX_dict()
##### Kernels
Klist = []
##### b_size
batch_size= 64

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


### Preprocessing

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import datetime

from preprocess import ProdKernel, FKernel, DictPrepross, Level_Normalizer
from utils import Load_FLX_dict, Plot_Batch
from generator import Diff_Generator, Up_and_Down_Generator

##### Dictionnary
D2 = []
D2 = [ DictPrepross(['o3','pl'], [Level_Normalizer(False),Level_Normalizer(True)] )]
D = [Load_FLX_dict()]
##### Kernels
Klist = []
##### Full Preprocessing :
FP = D + D2 + Klist
##### b_size
batch_size= 64

train_generator =  Up_and_Down_Generator(folder=data_folder, batch_size=batch_size, train=True, preprocess_x=FP)
validation_generator =  Up_and_Down_Generator(folder=data_folder, batch_size=batch_size, train=False, preprocess_x=FP, custom_b_p_e = 50)

## II) ARCHITECTURES

### Bidir RNN followed by fcn

In [4]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Activation, Flatten, Input, TimeDistributed
from keras.layers import Conv1D, UpSampling1D, AveragePooling1D, SeparableConv1D
from keras import optimizers
from keras.layers import Bidirectional
from keras import backend as K
from keras.losses import mean_squared_error
import keras

def one_loss(y_true, y_pred,i ):
    E = mean_squared_error(y_true[:,:,0], y_pred[:,:,0])
    return E

def flxd_loss(y_true, y_pred):
    E = mean_squared_error(y_true[:,:,0], y_pred[:,:,0])
    return E

def flxu_loss(y_true, y_pred):
    E = mean_squared_error(y_true[:,:,1], y_pred[:,:,1])
    return E

def dfdts_loss(y_true, y_pred, coef=50):
    E = mean_squared_error(coef*y_true[:,:,2], coef*y_pred[:,:,2])
    return E

def Total_loss(y_true, y_pred):
    E = flxd_loss(y_true, y_pred)
    E += flxu_loss(y_true, y_pred)
    E += dfdts_loss(y_true, y_pred)
    return(E)


n_channel = len(train_generator.variables)
o_channel = len(train_generator.new_variables_pred)

modelbd = Sequential()
modelbd.add(Bidirectional(LSTM(128, return_sequences=True, use_bias=False),input_shape=(72, n_channel)))
modelbd.add(Conv1D(50, use_bias=False,kernel_size=8 ,padding='same'))
modelbd.add(AveragePooling1D(7, padding='same', stride = 1 ))
modelbd.add(Activation('relu'))
modelbd.add(Conv1D(50, kernel_size=5 ,padding='same'))
modelbd.add(AveragePooling1D(4, padding='same', stride = 1 ))
modelbd.add(Activation('relu'))
modelbd.add(Conv1D(20, kernel_size=3 ,padding='same'))
modelbd.add(Activation('relu'))
modelbd.add(TimeDistributed(Dense(o_channel)))
#modelbd.add(Flatten())

M = Sequential()
M.add(UpSampling1D(5))
M.add(AveragePooling1D(26, padding='same', stride=5 ))
newInput = Input(shape=(72,11))
newOutputs  = M(newInput)
newOutputs2 = modelbd(newOutputs)
modelbd2 = keras.Model(newInput, newOutputs2)
rmsprop = optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=1.e-5)
modelbd2.compile(loss=Total_loss, optimizer=rmsprop,  metrics=[flxd_loss,flxu_loss, dfdts_loss])
modelbd2.summary()

modelbd.compile(loss='mse', optimizer=rmsprop)
prefix = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
archi = "Bidir"
modelbd.summary()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:39: UserWarning: Update your `AveragePooling1D` call to the Keras 2 API: `AveragePooling1D(7, padding="same", strides=1)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:42: UserWarning: Update your `AveragePooling1D` call to the Keras 2 API: `AveragePooling1D(4, padding="same", strides=1)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:51: UserWarning: Update your `AveragePooling1D` call to the Keras 2 API: `AveragePooling1D(26, padding="same", strides=5)`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 72, 11)            0         
_________________________________________________________________
sequential_2 (Sequential)    (None, 72, 11)            0         
_________________________________________________________________
sequential_1 (Sequential)    (None, 72, 3)             260369    
Total params: 260,369
Trainable params: 260,369
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 72, 256)           142336    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 72, 50)            102400    
_________________________________________________________________
aver

In [6]:
# Callbacks
class LossHistory(keras.callbacks.Callback):
    def __init__(self, frequency=1000):
        super(LossHistory, self).__init__()
        self.frequency=frequency

    @property
    def loss_name(self):
        return(['flxu_loss', 'flxd_loss', 'dfdts_loss', 'loss'])
        
    """Save the history of the loss """
    def on_train_begin(self, logs={}):
        self.losses = dict()
        for n in self.loss_name:
            self.losses[n] = []

    def on_batch_end(self, batch, logs={}):
        #print(logs['batch'])
        if(batch%self.frequency==0):
            for n in self.loss_name:
                self.losses[n].append( logs.get(n))
  
    def on_train_end(self, logs={}):
        for n in self.loss_name:
            self.losses[n] = np.array(self.losses[n])

LH = LossHistory(1000)

In [5]:
history = modelbd2.fit_generator(generator=train_generator ,
                    validation_data=validation_generator,
                             shuffle=False,
                               callbacks = [LH],
                               epochs=1,
                               verbose=1)

modelbd.save(  os.path.join(model_folder, prefix+archi+'.h5')  )

if(True):
    from contextlib import redirect_stdout
    with open(os.path.join(log_folder, prefix), 'w') as f:
        with redirect_stdout(f):
            modelbd2.summary()
            modelbd.summary()
            for i in FP:
                print(i)
            print(history)

NameError: name 'LH' is not defined

### ANALYSIS :

In [7]:
F = F_and

NameError: name 'F' is not defined

In [3]:
import matplotlib.pyplot as plt
f = plt.figure()
ax = plt.add_sp

### VISUALISATION

In [2]:
# FCT PLOT
def Show_triple_diff(y,y0):
    j = 50 #np.random.randint(0)
    f = plt.figure( figsize=(15,8)   )
    ax = plt.subplot(131)
    lev= 35
    plt.plot(np.flip(y[:,:,0].T[:lev,j]) , np.arange(lev));
    plt.plot(np.flip(y0[:,:,0].T[:lev,j]) , np.arange(lev));
    ax.legend(["truth", "pred"])
    ax = plt.subplot(132)
    plt.plot(np.flip(y[:,:,1].T[:,j]) , np.arange(72));
    plt.plot(np.flip(y0[:,:,1].T[:,j]) , np.arange(72));
    ax.legend(["truth", "pred"])
    ax = plt.subplot(133)
    plt.plot(np.flip(y[:,:,2].T[:,j]) , np.arange(72));
    plt.plot(np.flip(y0[:,:,2].T[:,j]) , np.arange(72));
    ax.legend(["truth", "pred"])



def Difference(y,y0):
#    y_cumsum = np.cumsum(y[i])
#    y0_cumsum = np.cumsum(y0)
    return(  np.mean(np.square(y-y0)))

def Compare(y,y0, i=0):
    f=plt.figure( figsize=(15,8), dpi=80)
    ax= f.add_subplot(1,2,1)
    ax.plot(np.flip(y0[i]), np.arange(len(y0[i]))) 
    ax.plot(np.flip(y[i]), np.arange(len(y0[i]))) 
    ax.legend(["y pred", 'y truth'])
#    ax.title("Diff")
    y_cumsum = np.cumsum(y[i])
    y0_cumsum = np.cumsum(y0[i])
    ax= f.add_subplot(1,2,2)
    ax.plot(np.flip(y0_cumsum), np.arange(len(y0[i])))        
    ax.plot(np.flip(y_cumsum), np.arange(len(y0[i])))        
    ax.legend(["y pred", 'y truth'])
#    ax.title("Cumulative")
    plt.show()

def eliminate_var(m,x):
    O = []
    for i in range(11):
        x0= x.copy()
        x0*=0
        x0[:,:,i]=x[:,:,i]
        O.append(modelbd.predict(x0))
    return(O)

def Plot_Predictions(O, y, header):
    f=plt.figure( figsize=(15,10), dpi=80)
    for i,y0 in enumerate(O):
        ax= f.add_subplot(3,4,i+1)
        ax.set_title(header[i])
        for b in range(y0.shape[0]):
            ax.plot(np.flip(y0[b]), np.arange(len(y0[b])))
    ax= f.add_subplot(3,4,12)
    ax.set_title('flx')
    for b in range(y0.shape[0]):
        ax.plot(np.flip(y[b]), np.arange(len(y[b])))

def Normal2(x,header):
    O1 = []#['fcld', 'q','qi','ql','rl','ri']
    N = [ 'pl']
    STD = []
    STD2 = []
    for i, h in enumerate(header):
        if h in O1:
            x[:,:,i] = np.max(x[:,:,i], axis=1).reshape(x.shape[0],1)
        if h in N:
            #print(h, np.mean(x[:,:,i], axis=0)[32])
            x[:,:,i] -= np.mean(x[:,:,i], axis=0)
        if h in STD:
            x[:,:,i] /= (x[:,-1,i]+0.000000001).reshape(-1,1)         
        if h in STD2:
            x[:,:,i] /= (x[:,0,i]+0.000000001).reshape(-1,1)         
    return(x)


### SECOND ARCHITECTURE :
Not tested yet

# III) TRACKS 

### MODEL 2 :

- FCNN
- (with AE)

##### MODEL 2 : FCM-Final FC

##### MODEL 2 : U-net :
- use regular U-net so all layers affect each other and more stability

### MODEL 3 : Bidir-LSTM 
> Possible alternatives

- use two LSTM to show both impact of superior and inferior layer
- use attention model over it
- use w embeddings before

> TD

- Read git trez
- Read article of Hedge fun